In [12]:
from numba import jit, int32, float64
import numpy as np

BUYER, INACTIVE, SELLER = 1, 0, -1

@jit(nopython=True)
def initialize_grid(width, height):
    """Initialize the market grid with all traders set to INACTIVE."""
    return np.full((height, width), INACTIVE, dtype=int32)

@jit(nopython=True)
def random_activation(grid, p):
    """Randomly activate a fraction p of traders in the grid."""
    height, width = grid.shape
    total_traders = height * width
    num_active_traders = int(p * total_traders)

    # Generate all indices and shuffle them
    flat_indices = np.arange(total_traders)
    np.random.shuffle(flat_indices)

    # Take the first `num_active_traders` indices
    selected_indices = flat_indices[:num_active_traders]

    for index in selected_indices:
        x, y = divmod(index, width)
        grid[x, y] = np.random.choice(np.array([BUYER, SELLER]))


@jit(nopython=True)
def get_neighbors(x, y, height, width):
    """Get the neighbors of a cell at position (x, y) using Von Neumann neighborhood."""
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    neighbors = []
    for dx, dy in directions:
        nx, ny = (x + dx) % height, (y + dy) % width
        neighbors.append((nx, ny))
    return neighbors

@jit(nopython=True)
def update_trader_states(grid, ph, pe, pd):
    """Apply state updates for all traders in the grid."""
    height, width = grid.shape
    updates = []

    for x in range(height):
        for y in range(width):
            current_state = grid[x, y]

            if current_state == INACTIVE:
                if np.random.random() < pe:
                    new_state = np.random.choice(np.array([BUYER, SELLER]))
                    updates.append((x, y, new_state))
            else:
                neighbors = get_neighbors(x, y, height, width)
                inactive_neighbors = [(nx, ny) for nx, ny in neighbors if grid[nx, ny] == INACTIVE]

                # Activate inactive neighbors with probability `ph`
                for nx, ny in inactive_neighbors:
                    if np.random.random() < ph:
                        new_state = np.random.choice(np.array([BUYER, SELLER]))
                        updates.append((nx, ny, new_state))

                # Deactivate the trader with probability proportional to inactive neighbors
                if np.random.random() < pd * len(inactive_neighbors):
                    updates.append((x, y, INACTIVE))

    for x, y, new_state in updates:
        grid[x, y] = new_state

@jit(nopython=True)
def run_percolation_dynamics(grid, steps, ph, pe, pd):
    """Simulate percolation dynamics for a number of steps."""
    for i in range(steps):
        update_trader_states(grid, ph, pe, pd)

# Example Usage
width, height = 512, 128
pd = 0.05
pe = 0.001
ph = 0.0485

# Initialize grid and market
market_grid = initialize_grid(width, height)
random_activation(market_grid, p=0.25)

# Run percolation dynamics
run_percolation_dynamics(market_grid, 100, ph, pe, pd)


In [6]:
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def animate_market(grid, ph, pe, pd, num_steps=100):
    """
    Animates the market dynamics over a given number of steps.

    Args:
        grid (np.ndarray): The grid representing the market states.
        ph (float): Probability to activate inactive neighbors.
        pe (float): Probability to activate an inactive trader.
        pd (float): Probability to deactivate a trader.
        num_steps (int): Number of steps to animate.

    Returns:
        HTML: HTML object containing the animation video.
    """
    # Initialize the grid states for animation
    grid_states = []

    # Set up the figure and axes for grid and line plots
    fig, (ax, ax_line) = plt.subplots(1, 2, figsize=(16, 8))

    # Initialize grid plot with the correct shape
    initial_state = grid.copy()
    grid_plot = ax.imshow(initial_state, cmap="coolwarm", vmin=-1, vmax=1)
    ax.set_title("Cellular Automata (Grid)")
    ax.axis("off")

    # Set up the line plot for tracking buyers, sellers, and total participants
    buyers_line, = ax_line.plot([], [], label="Buyers (1)", color="blue")
    sellers_line, = ax_line.plot([], [], label="Sellers (-1)", color="red")
    inactive_line, = ax_line.plot([], [], label="Inactive (0)", color="gray")

    ax_line.set_title("Market Participation Over Time")
    ax_line.set_xlim(0, num_steps)
    ax_line.set_ylim(0, grid.size)  # Max total participants
    ax_line.set_xlabel("Steps")
    ax_line.set_ylabel("Count")
    ax_line.legend()

    # Arrays to store counts over time
    buyers_counts = []
    sellers_counts = []
    inactive_counts = []

    def update(frame):
        """Update function for each animation frame."""
        nonlocal grid

        # Update the grid using the percolation dynamics
        update_trader_states(grid, ph, pe, pd)
        grid_states.append(grid.copy())
        grid_plot.set_array(grid)

        # Count the states
        buyers_count = np.sum(grid == 1)
        sellers_count = np.sum(grid == -1)
        inactive_count = grid.size - (buyers_count + sellers_count)

        # Update counts
        buyers_counts.append(buyers_count)
        sellers_counts.append(sellers_count)
        inactive_counts.append(inactive_count)

        # Update the line plots
        buyers_line.set_data(np.arange(len(buyers_counts)), buyers_counts)
        sellers_line.set_data(np.arange(len(sellers_counts)), sellers_counts)
        inactive_line.set_data(np.arange(len(inactive_counts)), inactive_counts)

        # Adjust y-axis limits dynamically
        max_count = max(max(buyers_counts), max(sellers_counts), max(inactive_counts))
        ax_line.set_ylim(0, max(max_count, 1.1 * grid.size))

        return grid_plot, buyers_line, sellers_line, inactive_line

    ani = FuncAnimation(fig, update, frames=num_steps, interval=200, blit=True)

    # Display the animation in the notebook
    plt.close(fig)  # Prevent duplicate static figures in notebooks
    return HTML(ani.to_html5_video())


In [13]:
pd = 0.05
pe = 0.001
ph = 0.0485
width, height = 50, 50
market_grid = initialize_grid(width, height)
random_activation(market_grid, p=0.25)
animate_market(market_grid, pe=pe, pd=pe, ph=ph, num_steps = 100)

In [9]:
market